# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [76]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [95]:
# 1)
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Município'
r = requests.get(url)
r.status_code
data_rj_json = r.json()
df = pd.DataFrame(data_rj_json['items'])
df.head(1)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,13753,Município,Piraí,3304003,RJ,None,Deferido,19405.000071/2002-58,2002-09-30T03:00:00Z,Operação contratual interna,PMAT,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,408600.0,0,0,11/10/2002


In [79]:
# 2)
df_top_status_freq = df['status'].value_counts().head(3)
df_top_status_freq

Deferido        94
Arquivado       53
Regularizado    25
Name: status, dtype: int64

In [96]:
df.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,13753,Município,Piraí,3304003,RJ,None,Deferido,19405.000071/2002-58,2002-09-30T03:00:00Z,Operação contratual interna,PMAT,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,408600.00,0,0,11/10/2002
1,13579,Município,Rio de Janeiro,3304557,RJ,None,Arquivado,19405.000029/2006-61,2006-07-20T03:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1316493.93,0,0,26/06/2007
2,11022,Município,Casimiro de Abreu,3301306,RJ,None,Deferido,17944.001338/2011-92,2012-04-18T03:00:00Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,3900000.00,1,0,29/06/2012
3,11053,Município,Macaé,3302403,RJ,None,Deferido,17944.001346/2011-39,2011-11-29T02:00:00Z,Operação contratual interna,PAC 2 - Pró-Moradia,Instituição Financeira Nacional,Caixa Econômica Federal,Real,48286602.68,1,0,08/12/2011
4,13229,Município,Nova Friburgo,3303401,RJ,None,Deferido,17944.001968/2006-08,2007-01-18T02:00:00Z,Operação contratual interna,Provias,Instituição Financeira Nacional,Banco do Brasil S/A,Real,1976972.00,0,0,15/02/2007


In [81]:
# 3)
ano = df['data_status'].str.slice(start=-4)
ano
#df['data_status'].head(1)

0      2002
1      2007
2      2012
3      2011
4      2007
       ... 
230    2021
231    2010
232    2010
233    2008
234    2013
Name: data_status, Length: 235, dtype: object

In [82]:
#4)
ano.value_counts()

2008    37
2007    29
2014    20
2010    19
2019    17
2020    15
2013    15
2022    14
2012    13
2011    10
2015     9
2009     9
2017     5
2021     5
2018     4
2002     4
2016     4
2006     4
2004     1
2003     1
Name: data_status, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [87]:
#1)
# https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Município
def monta_requisicao (uf: str, tp_interessado: str) -> pd.DataFrame:
    url_req = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf='+uf.upper()+'&tipo_interessado='+tp_interessado.capitalize()
    r = requests.get(url_req)
    data_json = r.json()
    df_req = pd.DataFrame(data_json['items'])
    return df_req

In [88]:
# 2)
df = monta_requisicao('mg','estado')
df[df['status']=='Arquivado por decurso de prazo']['status'].value_counts()


Arquivado por decurso de prazo    1
Name: status, dtype: int64

In [89]:
# 3)
df = monta_requisicao('ba','município')
df[df['status']=='Deferido']['interessado'].value_counts().head(1)

Luís Eduardo Magalhães    14
Name: interessado, dtype: int64

In [91]:
# 4) 
df = monta_requisicao('ba','estado')
df.to_csv('Requisições-BA.csv',sep=';',index=False)